In [2]:
import energyplus.ooep as ooep
import energyplus.ooep.ems
import energyplus.ooep.addons.state
import numpy as np
import pandas as pd


observations = []
keys = ['CHILLER', 'CHILLER 1', 'CHILLER 2']

for episode in range(1):  
    env = energyplus.ooep.ems.Environment().__enter__()
    P_ch = [env.variable(dict(
            variable_name='Chiller Electricity Rate',
            variable_key=key)) for key in keys]
    T_outdoor_dry = env.variable(dict(
            variable_name='Site Outdoor Air Drybulb Temperature',
            variable_key='ENVIRONMENT'))
    T_outdoor_wet = env.variable(dict(
            variable_name='Site Outdoor Air Wetbulb Temperature',
            variable_key='ENVIRONMENT'))
    T_chws = [env.variable(dict(
            variable_name='Chiller Evaporator Outlet Temperature', 
            variable_key=key)) for key in keys]
    T_chwr = [env.variable(dict(
            variable_name='Chiller Evaporator Inlet Temperature',  
            variable_key=key)) for key in keys]
    F_chw = [env.variable(dict(
            variable_name='Chiller Evaporator Mass Flow Rate',  
            variable_key=key)) for key in keys]
    T_chws_set = env.actuator(dict(
            component_type='System Node Setpoint',
            control_type = 'Temperature Setpoint', 
            actuator_key='CHW Loop Supply Side Outlet'))
    

    sm_env = ooep.addons.state.StateMachine(env)
    sm_stepf = sm_env.step_function(
        dict(event_name='begin_zone_timestep_after_init_heat_balance')
    )

    sm_env.run(
        '--output-directory', 'build3/demo-eplus',
        '--weather', f'./SGP_Singapore_486980_IWEC.epw',
        f'./jan4withEMS.idf',
        verbose=True,
    )

    while True:

        sm_stepf()


        try:

            P_ch = [env.variable(dict(
                        variable_name='Chiller Electricity Rate',
                        variable_key=key)) for key in keys]
            power = P_ch[0].value + P_ch[1].value + P_ch[2].value

            print(power)
            break

        except ooep.ems.NotReadyError: pass

    for step in range(1, 1000):
        
        obs = []

        # Add Chiller Electricity values to the observation space
        for chiller_electricity_rate in P_ch:
            obs.append(chiller_electricity_rate.value)


        obs.append(T_outdoor_dry.value)
        obs.append(T_outdoor_wet.value)

        for chiller_evaporator_outlet_temp in T_chws:
            obs.append(chiller_evaporator_outlet_temp.value)

        for chiller_evaporator_inlet_temp in T_chwr:
            obs.append(chiller_evaporator_inlet_temp.value)
         
        for chiller_evaporator_mass_flow_rate in F_chw:
            obs.append(chiller_evaporator_mass_flow_rate.value)

        
        observations.append(obs)
        
        sm_stepf()

        print(rf"Episode={episode+1} Step={step}  P_ch={P_ch[0].value,P_ch[1].value,P_ch[2].value}) (T_outdoor_dry={T_outdoor_dry.value}) (T_outdoor_wet={T_outdoor_wet.value}) ", flush=True, file=__import__('sys').stderr)


Episode=1 Step=1  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=25.000000000000004) (T_outdoor_wet=23.75416742323046) 
Episode=1 Step=2  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.9) (T_outdoor_wet=23.819521417226728) 
Episode=1 Step=3  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.9) (T_outdoor_wet=23.819521417226728) 
Episode=1 Step=4  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.799999999999997) (T_outdoor_wet=23.88316348871006) 
Episode=1 Step=5  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.799999999999997) (T_outdoor_wet=23.88316348871006) 
Episode=1 Step=6  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.7) (T_outdoor_wet=23.945140943508445) 
Episode=1 Step=7  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.7) (T_outdoor_wet=23.945140943508445) 
Episode=1 Step=8  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.625) (T_outdoor_wet=23.871365193710023) 
Episode=1 Step=9  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.625) (T_outdoor_wet=23.871365193710023) 
Episode=1 Step=10  P_ch=(0.0, 0.0, 0.0)) (T_outdoor_dry=24.549999999999997) (T_outdo

0.0


Episode=1 Step=61  P_ch=(2516.761478424981, 0.0, 0.0)) (T_outdoor_dry=24.275000000000002) (T_outdoor_wet=23.3366844334709) 
Episode=1 Step=62  P_ch=(3723.9147235366513, 0.0, 0.0)) (T_outdoor_dry=24.3) (T_outdoor_wet=23.29743442618315) 
Episode=1 Step=63  P_ch=(3723.9147235366513, 0.0, 0.0)) (T_outdoor_dry=24.3) (T_outdoor_wet=23.29743442618315) 
Episode=1 Step=64  P_ch=(4475.44868176085, 0.0, 0.0)) (T_outdoor_dry=24.6) (T_outdoor_wet=23.397171804990755) 
Episode=1 Step=65  P_ch=(4475.44868176085, 0.0, 0.0)) (T_outdoor_dry=24.6) (T_outdoor_wet=23.397171804990755) 
Episode=1 Step=66  P_ch=(78007.59206916002, 0.0, 0.0)) (T_outdoor_dry=24.9) (T_outdoor_wet=23.492859911794824) 
Episode=1 Step=67  P_ch=(78007.59206916002, 0.0, 0.0)) (T_outdoor_dry=24.9) (T_outdoor_wet=23.492859911794824) 
Episode=1 Step=68  P_ch=(65163.158361633956, 0.0, 28699.764886154546)) (T_outdoor_dry=25.2) (T_outdoor_wet=23.58440849693524) 
Episode=1 Step=69  P_ch=(65163.158361633956, 0.0, 28699.764886154546)) (T_outdo

In [7]:
# Convert observations list to DataFrame
columns = ['Datetime','Power1','FOG1','MSG1', 'Power2','FOG2','MSG2', 'Power3','FOG3','MSG3', 'T_outdoor_dry', 'T_outdoor_wet', 'T_chws1', 'T_chws2', 'T_chws3', 'T_chwr1', 'T_chwr2', 'T_chwr3', 'F_chw1', 'F_chw2', 'F_chw3']
df_observations = pd.DataFrame(observations, columns=columns)

# Save the DataFrame to an Excel file
df_observations.to_excel('observations6.xlsx', index=False)

ValueError: 21 columns passed, passed data had 14 columns

In [11]:
from sklearn.ensemble import RandomForestRegressor

# Read the Excel file into a DataFrame
columns = ['Power1', 'Power2', 'Power3', 'T_outdoor_dry', 'T_outdoor_wet', 'T_chws1', 'T_chws2', 'T_chws3', 'T_chwr1', 'T_chwr2', 'T_chwr3', 'F_chw1', 'F_chw2', 'F_chw3']

df_observations = pd.DataFrame(observations, columns=columns)


# Handle missing values or inconsistencies in the data if needed
df = df_observations.dropna()

# Create a new column for the step number
df['Step'] = range(1, len(df) + 1)

# Create the 'FOG' and 'MSG' features for each chiller
for chiller in range(1, 4):
    df[f'FOG{chiller}'] = df[f'Power{chiller}'].diff(periods=5)
    df[f'MSG{chiller}'] = df[f'Power{chiller}'].diff().rolling(window=5).apply(lambda x: np.mean(x ** 2))

# Save the DataFrame back to the Excel file
df.to_excel('observations4.xlsx', index=False)

# Specify the features to be used as input including Month, Day, Hour
features1 = ['F_chw1', 'T_chws1', 'T_chwr1', 'T_outdoor_dry', 'T_outdoor_wet', 'FOG1', 'MSG1']
X1 = df[features1]

features2 = ['F_chw2', 'T_chws2', 'T_chwr2', 'T_outdoor_dry', 'T_outdoor_wet', 'FOG2', 'MSG2']
X2 = df[features2]

features3 = ['F_chw3', 'T_chws3', 'T_chwr3', 'T_outdoor_dry', 'T_outdoor_wet', 'FOG3', 'MSG3']
X3 = df[features3]

# Initialize the Random Forest models with parameters from the paper
rf_model1 = RandomForestRegressor(n_estimators=100, random_state=42, max_features='sqrt')
rf_model2 = RandomForestRegressor(n_estimators=100, random_state=42, max_features='sqrt')
rf_model3 = RandomForestRegressor(n_estimators=100, random_state=42, max_features='sqrt')

# Train the models
rf_model1.fit(X1, df['Power1'].shift(-1).fillna(df['Power1']))
rf_model2.fit(X2, df['Power2'].shift(-1).fillna(df['Power2']))
rf_model3.fit(X3, df['Power3'].shift(-1).fillna(df['Power3']))

# Make predictions
df['Prediction_RF_1 (Power1+1)'] = rf_model1.predict(X1)
df['Prediction_RF_2 (Power2+1)'] = rf_model2.predict(X2)
df['Prediction_RF_3 (Power3+1)'] = rf_model3.predict(X3)





In [13]:
# Combine original observations DataFrame with predictions
df_predictions = df.copy()  # Create a copy of the DataFrame to avoid modifying the original
df_predictions['Prediction_RF_1 (Power1+1)'] = rf_model1.predict(X1)
df_predictions['Prediction_RF_2 (Power2+1)'] = rf_model2.predict(X2)
df_predictions['Prediction_RF_3 (Power3+1)'] = rf_model3.predict(X3)

# Specify the columns to include in the final Excel file
columns_to_include = [
    'Step','F_chw1', 'T_chws1', 'T_chwr1', 'Power1', 'FOG1', 'MSG1', 'Prediction_RF_1 (Power1+1)',
    'F_chw2', 'T_chws2', 'T_chwr2', 'Power2', 'FOG2', 'MSG2', 'Prediction_RF_2 (Power2+1)',
    'F_chw3', 'T_chws3', 'T_chwr3', 'Power3', 'FOG3', 'MSG3', 'Prediction_RF_3 (Power3+1)',
    'T_outdoor_dry', 'T_outdoor_wet'
]

# Filter the DataFrame to include only the specified columns
df_final = df_predictions[columns_to_include]

# Write the DataFrame to an Excel file
df_final.to_excel('observations_and_predictions.xlsx', index=False)


In [16]:
# Combine original observations DataFrame with predictions
df_predictions = df.copy()  # Create a copy of the DataFrame to avoid modifying the original
df_predictions['Prediction_RF_1 (Power1+1)'] = rf_model1.predict(X1)
df_predictions['Prediction_RF_2 (Power2+1)'] = rf_model2.predict(X2)
df_predictions['Prediction_RF_3 (Power3+1)'] = rf_model3.predict(X3)

# Specify the columns to include in the final Excel file
columns_to_include_in_power_predictions = [
    'Step', 'Prediction_RF_1 (Power1+1)', 'Prediction_RF_2 (Power2+1)', 'Prediction_RF_3 (Power3+1)'
]

# Filter the DataFrame to include only the specified columns
df_power_predictions = df_predictions[columns_to_include_in_power_predictions]

# Write the DataFrame to an Excel file
df_power_predictions.to_excel('PowerPredictions.xlsx', index=False)


In [20]:
# Create a DataFrame containing only the power values and their predictions
df_power_predictions = pd.DataFrame({
    'Power1': df['Power1'],
    'Prediction_RF_1 (Power1+1)': rf_model1.predict(X1),
    'Power2': df['Power2'],
    'Prediction_RF_2 (Power2+1)': rf_model2.predict(X2),
    'Power3': df['Power3'],
    'Prediction_RF_3 (Power3+1)': rf_model3.predict(X3)
})

# Write the DataFrame to an Excel file
df_power_predictions.to_excel('New_power_and_predictions.xlsx', index=False)
